<a href="https://colab.research.google.com/github/RybakovAlex/Project-Python/blob/main/%D0%91%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82_%22GB_Homework_3_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Урок 3. Работа с библиотекой Pandas, часть 2. Манипуляции с данными


1. Использовать датасет с текущего урока и построить сегментированную воронку конверсии пользователей интернет-магазина по полу.

In [113]:
# Импортируем нужную бибилиотеку
import pandas as pd

In [114]:
# Загружаем в Colab все датасеты
user_table = pd.read_csv('/content/user_table.csv')
user_table.head()

,user_id,date,device,sex
0,450007,2015-02-28,Desktop,Female
1,756838,2015-01-13,Desktop,Male
2,568983,2015-04-09,Desktop,Male
3,190794,2015-02-18,Desktop,Female
4,537909,2015-01-15,Desktop,Male


In [115]:
user_table.shape

(90400, 4)

In [116]:
home_page = pd.read_csv('/content/home_page_table.csv')
home_page.head()

,user_id,page
0,313593,home_page
1,468315,home_page
2,264005,home_page
3,290784,home_page
4,639104,home_page


In [117]:
home_page.shape

(90400, 2)

In [118]:
search_page = pd.read_csv('/content/search_page_table.csv')
search_page.head()

,user_id,page
0,15866,search_page
1,347058,search_page
2,577020,search_page
3,780347,search_page
4,383739,search_page


In [119]:
search_page.shape

(45200, 2)

In [120]:
payment_page = pd.read_csv('/content/payment_page_table.csv')
payment_page.head()

,user_id,page
0,253019,payment_page
1,310478,payment_page
2,304081,payment_page
3,901286,payment_page
4,195052,payment_page


In [121]:
payment_page.shape

(6030, 2)

In [122]:
payment_confirmation = pd.read_csv('/content/payment_confirmation_table.csv')
payment_confirmation.head()

,user_id,page
0,123100,payment_confirmation_page
1,704999,payment_confirmation_page
2,407188,payment_confirmation_page
3,538348,payment_confirmation_page
4,841681,payment_confirmation_page


In [123]:
payment_confirmation.shape

(452, 2)

In [124]:
# Меняем название одинаковой колонки в каждом датасете"
home_page = home_page.rename(columns = {'page':'home_page'})
search_page = search_page.rename(columns = {'page':'search_page'})
payment_page = payment_page.rename(columns = {'page':'payment_page'})
payment_confirmation = payment_confirmation.rename(columns = {'page':'payment_confirmation'})

In [125]:
# Склеиваем последовательно все датасеты с помощью метода merge по идентификатору user_id"
df = user_table.merge(home_page, on = 'user_id', how = 'left').merge(search_page, on = 'user_id', how = 'left') \
.merge(payment_page, on = 'user_id', how = 'left').merge(payment_confirmation, on = 'user_id', how = 'left')
df.head()

,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation
0,450007,2015-02-28,Desktop,Female,home_page,NaN,NaN,NaN
1,756838,2015-01-13,Desktop,Male,home_page,NaN,NaN,NaN
2,568983,2015-04-09,Desktop,Male,home_page,search_page,NaN,NaN
3,190794,2015-02-18,Desktop,Female,home_page,search_page,NaN,NaN
4,537909,2015-01-15,Desktop,Male,home_page,NaN,NaN,NaN


In [126]:
df.shape

(90400, 8)

In [127]:
# Второй способ склеевания: склеиваем все датасеты сразу
from functools import reduce
data_frames = [user_table, home_page, search_page, payment_page, payment_confirmation]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on='user_id', how='left'), data_frames)
df_merged.head()

,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation
0,450007,2015-02-28,Desktop,Female,home_page,NaN,NaN,NaN
1,756838,2015-01-13,Desktop,Male,home_page,NaN,NaN,NaN
2,568983,2015-04-09,Desktop,Male,home_page,search_page,NaN,NaN
3,190794,2015-02-18,Desktop,Female,home_page,search_page,NaN,NaN
4,537909,2015-01-15,Desktop,Male,home_page,NaN,NaN,NaN


In [128]:
# Для удоства последующих расчетов, перекодируем названия событий в 0/1 с помощью лямбда-функции
df['home_page'] = df['home_page'].apply(lambda x: 1 if x == 'home_page' else 0)
df['search_page'] = df['search_page'].apply(lambda x: 1 if x == 'search_page' else 0)
df['payment_page'] = df['payment_page'].apply(lambda x: 1 if x == 'payment_page' else 0)
df['payment_confirmation'] = df['payment_confirmation'].apply(lambda x: 1 if x == 'payment_confirmation_page' else 0)
df.head()

,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation
0,450007,2015-02-28,Desktop,Female,1,0,0,0
1,756838,2015-01-13,Desktop,Male,1,0,0,0
2,568983,2015-04-09,Desktop,Male,1,1,0,0
3,190794,2015-02-18,Desktop,Female,1,1,0,0
4,537909,2015-01-15,Desktop,Male,1,0,0,0


In [129]:
# Агрегируем данные: подсчитываем количество пользователей по разным типам устройств
df['device'].value_counts()

Desktop    60200
Mobile     30200
Name: device, dtype: int64

In [130]:
# Агрегируем данные: подсчитываем общее количество пользователей и сколько из них подтвердили покупку
df['payment_confirmation'].agg({'users':'count', 'payers':'sum'})

users     90400
payers      452
Name: payment_confirmation, dtype: int64

In [131]:
# Агрегируем данные: подсчитываем сколько пользователей совершило каждое из 4х событий на сайте\n",
summary = df[['home_page', 'search_page', 'payment_page', 'payment_confirmation']].sum().reset_index()
summary.head()


,index,0
0,home_page,90400
1,search_page,45200
2,payment_page,6030
3,payment_confirmation,452


In [132]:
# Для удобства переименуем столбцы
summary = summary.rename(columns = {'index':'stage', 0:'number'})
summary.head()

,stage,number
0,home_page,90400
1,search_page,45200
2,payment_page,6030
3,payment_confirmation,452


In [133]:
# Импортируем графическую библиотеку Plotly (https://plotly.com/python/funnel-charts/)
from plotly import graph_objects as go

In [141]:
# Настраиваем скрипт этой библиотеки для построения воронки конверсий нашими параметрами
fig = go.Figure(go.Funnel(
y = summary['stage'],
x = summary['number'],
textinfo = 'value + percent previous' # значение + процент от предыдущего
))

fig.update_layout(
        title = 'Воронка конверсии Интернет-магазина',
        title_x=0.5,
        autosize=False,
        width=1200,
        height=600)

fig.show()

2. Создать 2 новые фичи на основе колонки “date”: месяц и день недели (пример https://stackoverflow.com/a/62624729, https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.weekday.html).

3. Определить самый топовый по продажам месяц и день недели с помощью базовых методов агрегации (sum или count).

4. Выяснить, в какой день недели лучше покупают женщины, а в какой мужчины с помощью одного из методов построения сводных таблиц (groupby, pivot_table или crosstab).